In [1]:
#import data
import pandas as pd
import altair as alt
import io
import matplotlib.pyplot as plt
import seaborn as sns, numpy as np
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# from google.colab import files
# uploaded = files.upload()

Store in aniList


In [ ]:
# aniList = pd.read_csv(io.BytesIO(uploaded['AnimeList.csv']))
url = 'https://raw.githubusercontent.com/Ninelie-Nananai/AnimeListApi/main/AnimeList.csv'
aniList = pd.read_csv(url)
aniCopy = aniList.copy()

In [ ]:
aniList.isna().sum()

anime_id              0
title                 0
title_english      8754
title_japanese       35
title_synonyms     5541
image_url            96
type                  0
source                0
episodes              0
status                0
airing                0
aired_string          0
aired                 0
duration              0
rating                0
score                 0
scored_by             0
rank               1574
popularity            0
members               0
favorites             0
background        13421
premiered         10382
broadcast         10207
related               0
producer           6190
licensor          11105
studio             5934
genre                64
opening_theme         0
ending_theme          0
dtype: int64

In [ ]:
# columns = ['title','genre','title_synonyms']
# for col in columns:
# aniList[col].apply(lambda x: x.strip())

In [ ]:
aniList.column = aniList.columns.str.strip()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



In [ ]:
def combine_string(row):
  res = [str(r) for r in row]
  return ' | '.join(res)

In [ ]:
aniList.head(5)

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,airing,aired_string,aired,duration,rating,score,scored_by,rank,popularity,members,favorites,background,premiered,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,False,"Jan 13, 2012 to Mar 30, 2012","{'from': '2012-01-13', 'to': '2012-03-30'}",24 min. per ep.,PG-13 - Teens 13 or older,7.63,139250,1274.0,231,283882,2809,Inu x Boku SS was licensed by Sentai Filmworks...,Winter 2012,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ..."
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,False,"Apr 2, 2007 to Oct 1, 2007","{'from': '2007-04-02', 'to': '2007-10-01'}",24 min. per ep.,PG-13 - Teens 13 or older,7.89,91206,727.0,366,204003,2579,NaN,Spring 2007,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi..."
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,False,"Oct 4, 2008 to Sep 25, 2009","{'from': '2008-10-04', 'to': '2009-09-25'}",24 min. per ep.,PG - Children,7.55,37129,1508.0,1173,70127,802,NaN,Fall 2008,Unknown,"{'Adaptation': [{'mal_id': 101, 'type': 'manga...","TV Tokyo, Sotsu",NaN,Satelight,"Comedy, Magic, School, Shoujo","['#1: ""Minna no Tamago (みんなのたまご)"" by Shugo Cha...","['#1: ""Rottara Rottara (ロッタラ ロッタラ)"" by Buono! ..."
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,False,"Aug 16, 2002 to May 23, 2003","{'from': '2002-08-16', 'to': '2003-05-23'}",16 min. per ep.,PG-13 - Teens 13 or older,8.21,36501,307.0,916,93312,3344,Princess Tutu aired in two parts. The first pa...,Summer 2002,Fridays at Unknown,"{'Adaptation': [{'mal_id': 1581, 'type': 'mang...","Memory-Tech, GANSIS, Marvelous AQL",ADV Films,Hal Film Maker,"Comedy, Drama, Magic, Romance, Fantasy","['""Morning Grace"" by Ritsuko Okazaki']","['""Watashi No Ai Wa Chiisaikeredo"" by Ritsuko ..."
4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,False,"Oct 6, 2012 to Mar 30, 2013","{'from': '2012-10-06', 'to': '2013-03-30'}",24 min. per ep.,PG-13 - Teens 13 or older,8.67,107767,50.0,426,182765,2082,NaN,Fall 2012,Unknown,"{'Adaptation': [{'mal_id': 9711, 'type': 'mang...","NHK, Shueisha",NaN,J.C.Staff,"Comedy, Drama, Romance, Shounen","['#1: ""Moshimo no Hanashi (もしもの話)"" by nano.RIP...","['#1: ""Pride on Everyday"" by Sphere (eps 1-13)..."


In [ ]:
combine_columns = ['title','genre','title_synonyms']
aniList['combined'] = aniList[combine_columns].apply(combine_string, axis=1)

In [ ]:
aniList['combined'][0]

'Inu x Boku SS | Comedy, Supernatural, Romance, Shounen | Youko x Boku SS'

In [ ]:
aniList['combined'][1]

'Seto no Hanayome | Comedy, Parody, Romance, School, Shounen | The Inland Sea Bride'

In [ ]:
text_vector = CountVectorizer().fit_transform(aniList['combined'])

In [ ]:
text_similarity = cosine_similarity(text_vector)

In [ ]:
print(aniList[['anime_id','title']].head().values)

[[11013 'Inu x Boku SS']
 [2104 'Seto no Hanayome']
 [5262 'Shugo Chara!! Doki']
 [721 'Princess Tutu']
 [12365 'Bakuman. 3rd Season']]


In [ ]:
def get_similar(anime_id):
  DISPLAY = 'combined'
  # get index from anime id

  index = aniList.loc[aniList['anime_id']== anime_id].index.values[0]

  #pair anime id ,title, score

  id_score  = [(aniList.loc[id,'anime_id'], aniList.loc[id,DISPLAY], sim) for id, sim in enumerate(text_similarity[index,:])]

  # sort list by score

  sorted_score = sorted(id_score,key=lambda x:x[-1], reverse=True)
  # return selected anime

  return aniList.loc[aniList['anime_id']==anime_id,DISPLAY].values[0], pd.DataFrame(sorted_score[1:5+1],columns=['anime_id',DISPLAY,'score'])


In [ ]:
print(get_similar(anime_id = 2104  ))

('Seto no Hanayome | Comedy, Parody, Romance, School, Shounen | The Inland Sea Bride',    anime_id                                           combined     score
0      4163  Seto no Hanayome OVA | Comedy, Parody, Romance...  0.643596
1      5199  Seto no Hanayome Fan Disc | Comedy, Parody, Sc...  0.577350
2      7041  Seto no Hanayome OVA Specials | Comedy, Parody...  0.566947
3      1452  The Samurai | Comedy, Romance, School, Shounen...  0.545545
4        24  School Rumble | Comedy, Romance, School, Shoun...  0.481125)


In [ ]:
!pip install -q gradio

     |████████████████████████████████| 979 kB 5.1 MB/s 
     |████████████████████████████████| 206 kB 59.3 MB/s 
     |████████████████████████████████| 1.9 MB 35.7 MB/s 
     |████████████████████████████████| 3.6 MB 69.3 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 961 kB 50.7 MB/s 


In [ ]:
np.random.choice(list(aniList['anime_id'].unique()))

8956

In [ ]:
import gradio as gr
iface = gr.Interface(fn=get_similar,
                     inputs=[gr.inputs.Number(label='Enter Anime ID')],
                     outputs=[gr.outputs.Textbox(label='Anime Title'),gr.outputs.Dataframe(label='Anime List')])
iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://26388.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://26388.gradio.app')